In [ ]:
# Imports 

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, ElasticNet
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error

In [ ]:
class DataSet:
    TRAINING_DIRECTORY = "train_files/"
    TEST_DIRECTORY = "example_test_files/"
    
    def __init__(self, seed = 42, train_size = 0.83):
        # Feel free to add the other .csv files here as well. 
        self._trades = pd.read_csv(self.TRAINING_DIRECTORY + "trades.csv")
        self._stock_list = pd.read_csv("stock_list.csv")
        self._stock_prices = pd.read_csv(
            self.TRAINING_DIRECTORY + "stock_prices.csv", 
            converters = { "ExpectedDividend": lambda x: 1 if x == "True" else 0 })
        
        self.seed = seed
        self.train_size = train_size
        self.has_split = False 
       
        self.X_train = dict()
        self.Y_train = dict()
        self.X_test = dict()
        self.Y_test = dict()
     
    def stocks(self) -> np.ndarray:
        return self._stock_prices["SecuritiesCode"].unique()
     
    def __split(self):
        if self.has_split:
            return 
        
        security_codes = self._stock_prices["SecuritiesCode"].unique()        
        for security_code in security_codes:
            stock_info = self._stock_prices[self._stock_prices["SecuritiesCode"] == security_code]
            features = stock_info.drop(["Target", "Date", "RowId"], axis = 1).to_numpy(dtype='float', na_value = 0)
            target = stock_info["Target"].to_numpy(dtype='float', na_value = 0)
            
            curr_x_train, curr_x_test, curr_y_train, curr_y_test = train_test_split(features, target, train_size = self.train_size, random_state = self.seed)
            self.X_train[security_code] = np.array(curr_x_train)
            self.X_test[security_code] = np.array(curr_x_test)
            self.Y_train[security_code] = np.array(curr_y_train)
            self.Y_test[security_code] = np.array(curr_y_test)
            
        self.has_split = True 
    
    def _collate_data(self, values) -> np.ndarray:
        res = np.array([])
        for value in values:
            res = np.concatenate((res, value))
        return res
    
    # Using methods below without specifying a security is a bit ugly...
    def x_train(self, security = None) -> np.ndarray:
        if not self.has_split:
            self.__split()
            
        if security is not None:
            return self.X_train[security]
        
        return self._collate_data(self.X_train.values())
        
    def x_test(self, security = None) -> np.ndarray:
        if not self.has_split:
            self.__split()
            
        if security is not None:
            return self.X_test[security]            
        
        return self._collate_data(self.X_test.values())

    def y_train(self, security = None) -> np.ndarray:
        if not self.has_split:
            self.__split()
            
        if security is not None:
            return self.Y_train[security]   
        
        return self._collate_data(self.Y_train.values())
            
    def y_test(self, security = None) -> np.ndarray:
        if not self.has_split:
            self.__split()
            
        if security is not  None:
            return self.Y_test[security]
        
        return self._collate_data(self.Y_test.values())
        
    # Use below if you wish to get the raw DataFrames 
    def stock_list(self) -> pd.DataFrame:
        return self._stock_list
    
    def trades(self) -> pd.DataFrame:
        return self._trades
    
    def stock_prices(self) -> pd.DataFrame:
        return self._stock_prices

ds = DataSet()

In [ ]:
def plot_stock_price(securities_code):
    df = ds.stock_prices()
    df[df["SecuritiesCode"] == securities_code].plot(x = "Date", y = "Close", figsize=(8,4))
    plt.ylabel("Close")
    plt.title("Close for {}".format(securities_code))
    plt.locator_params(axis='x', nbins=8)
    plt.show()

def plot_stock_target(securities_code):
    df = ds.stock_prices()
    df[df["SecuritiesCode"] == securities_code].plot(x = "Date", y = "Target", figsize=(8,4))
    plt.ylabel("Target")
    plt.title("Target for {}".format(securities_code))
    plt.locator_params(axis='x', nbins=8)
    plt.show()

plot_stock_price(6502)
plot_stock_target(6502)


In [ ]:
# Matt's Linear Models

class BasicModels:
    def __init__(self):
        self.basic_features = [
            "SecuritiesCode",
            "Close",
            "Volume",
            "ExpectedDividend",
        ]
        self.ds = DataSet()
        self.lr_models = dict()
        self.rg_models = dict()
        self.en_models = dict()
        self.trained = False

      
    def train(self):
        if self.trained:
            return 
        
        for security_code in self.ds.stocks():
            x_train = self.ds.x_train(security_code)
            y_train = self.ds.y_train(security_code)
            
            cv_alpha = make_pipeline(StandardScaler(), RidgeCV(alphas = np.logspace(-6, 6, 13))).fit(x_train, y_train).named_steps['ridgecv'].alpha_
            
            lr_model = LinearRegression().fit(x_train, y_train)
            rg_model = make_pipeline(StandardScaler(), Ridge(alpha = cv_alpha)).fit(x_train, y_train)
            en_model = make_pipeline(StandardScaler(), ElasticNet()).fit(x_train, y_train)
            
            self.lr_models[security_code] = lr_model
            self.rg_models[security_code] = rg_model
            self.en_models[security_code] = en_model
        
        self.trained = True 
        
    def _compute_model_loss(self, models):
        mse_sum = 0
        for security_code in self.ds.stocks():
            x_test = self.ds.x_test(security_code)
            y_test = self.ds.y_test(security_code)
            
            lr_model = models[security_code]
            y_pred = lr_model.predict(x_test)
            
            mse = mean_squared_error(y_test, y_pred)
            mse_sum += mse
        return mse_sum
        
    def compute_model_losses(self):
        if self.trained == False:
            self.train()
    
        lr_loss = self._compute_model_loss(self.lr_models)
        rg_loss = self._compute_model_loss(self.rg_models)
        en_loss = self._compute_model_loss(self.en_models)
            
        print(f"Linear Regression Loss: {lr_loss}")
        print(f"Ridge Regression Loss: {rg_loss}")
        print(f"Elastic Net Loss: {en_loss}")
        
    def plot_results(self):
        for security_code in self.ds.stocks():
            x_test = self.ds.x_test(security_code)
            y_test = self.ds.y_test(security_code)
            
            lr_model = self.rg_models[security_code]
            lr_y_pred = lr_model.predict(x_test)
            
            rg_model = self.rg_models[security_code]
            rg_y_pred = rg_model.predict(x_test)
            
            en_model = self.en_models[security_code]
            en_y_pred = en_model.predict(x_test)
            
            fig, ax = plt.subplots()
            ax.scatter(y_test, lr_y_pred, label = "Linear Regression")
            ax.scatter(y_test, rg_y_pred, label = "Ridge Regression")
            ax.scatter(y_test, en_y_pred, label = "Elastic Net")
            
            ax.legend() 
            
            plt.xlabel("True Target")
            plt.ylabel("Predicted Target")
            plt.title("Predictions for {}".format(security_code))
            plt.axis('equal')
            plt.show()
            return 
        
bm = BasicModels()
bm.compute_model_losses()
bm.plot_results()
